In [7]:
import PyPDF2
import chromadb
from sentence_transformers import SentenceTransformer
import os

model = SentenceTransformer('all-MiniLM-L6-v2')
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="pdf_documents")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# !pip install PyPDF2 chromadb sentence_transformers

In [8]:

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text() + "\n"
    return text

In [9]:

def chunk_text(text, chunk_size=500, overlap=50):
    """Split text into overlapping chunks"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)

        if i + chunk_size >= len(words):
            break

    return chunks


In [10]:

def process_pdf_folder(pdf_folder):
    """Process all PDFs in a folder"""
    documents = []
    metadatas = []
    ids = []

    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            print(f"Processing: {filename}")

            text = extract_text_from_pdf(pdf_path)

            chunks = chunk_text(text)

            for i, chunk in enumerate(chunks):
                doc_id = f"{filename}_{i}"
                documents.append(chunk)
                metadatas.append({
                    "source": filename,
                    "chunk_id": i,
                    "total_chunks": len(chunks)
                })
                ids.append(doc_id)

    if documents:
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print(f"Added {len(documents)} chunks to database")



In [12]:

pdf_folder = "/content"
process_pdf_folder(pdf_folder)

Processing: paper11.pdf


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 28.8MiB/s]


Added 13 chunks to database


In [13]:

results = collection.get(include=['embeddings', 'documents', 'metadatas'])


In [15]:
print("Collection contents:")
print(f"Total documents: {len(results['ids'])}")
print(f"Embedding dimension: {len(results['embeddings'][0]) if len(results['embeddings']) > 0 else 0}")

Collection contents:
Total documents: 13
Embedding dimension: 384


In [16]:
print("\nFirst 5 embeddings with their documents:")
for i in range(min(5, len(results['ids']))):
    print(f"\nDocument {i+1}:")
    print(f"ID: {results['ids'][i]}")
    print(f"Source: {results['metadatas'][i]['source']}")
    print(f"Text preview: {results['documents'][i][:100]}...")
    print(f"Embedding (first 5 values): {results['embeddings'][i][:5]}...")
    print(f"Embedding length: {len(results['embeddings'][i])}")


First 5 embeddings with their documents:

Document 1:
ID: paper11.pdf_0
Source: paper11.pdf
Text preview: AgCyRAG: an Agentic Knowledge Graph based RAG Framework for Automated Security Analysis Kabul Kurnia...
Embedding (first 5 values): [-0.02450012  0.00981242 -0.0806575  -0.00252891  0.0252237 ]...
Embedding length: 384

Document 2:
ID: paper11.pdf_1
Source: paper11.pdf
Text preview: assist analysts with tasks such as identifying suspicious behavior, correlating incidents, and reaso...
Embedding (first 5 values): [-0.06596904  0.02045585 -0.03807443  0.03637905  0.04063749]...
Embedding length: 384

Document 3:
ID: paper11.pdf_2
Source: paper11.pdf
Text preview: and empirical evidence. We demonstrate the effectiveness of AgCyRAG by means of three real-world cyb...
Embedding (first 5 values): [-0.03780096  0.03129927 -0.10914202 -0.03416102  0.0447034 ]...
Embedding length: 384

Document 4:
ID: paper11.pdf_3
Source: paper11.pdf
Text preview: more specialized KGs, such as the ICS-SEC